# Import module 

this code is implementation of "Improved Techniques for Training GANs", but I didn't find a better implementation to reproduce the result of the generator. I'll be back to refine this notebook!

the network structure borrow from https://github.com/LDOUBLEV/semi-supervised-GAN

In [1]:
import os
import math
import scipy
import random
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Loading MNIST data

In [3]:
os.environ['CUDA_VISIBLE_DEVICES']='1'
mnist = input_data.read_data_sets("data/mnist",one_hot=True)

Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz


# Helper Functions

In [4]:
def bn(net,scope,is_training):
    return tf.contrib.layers.batch_norm(net, decay=0.9,updates_collections=None, epsilon=1e-5,scale=True,
                                            is_training=is_training, scope=scope)

def conv(net, wscope, bscope, output_depth=64, receptive_field=[5,5], stride=[2,2], padding='SAME', isDiscriminator=False):
    shape = net.get_shape()
    weights = tf.get_variable(wscope, receptive_field+[shape[-1], output_depth],initializer=tf.truncated_normal_initializer(stddev=0.02))
    if isDiscriminator:
        tf.add_to_collection('regularizer', tf.contrib.layers.l2_regularizer(scale=0.0001)(weights)) # weight regularizer
    biases = tf.get_variable(bscope, [output_depth], initializer=tf.constant_initializer(0.0))
    net = tf.nn.conv2d(net, weights, strides=[1]+stride+[1], padding=padding)
    net = tf.reshape(tf.nn.bias_add(net, biases), net.get_shape())
    return net

def linear(net,wscope,bscope,output_depth,isDiscriminator=False):
    shape = net.get_shape()       
    weights = tf.get_variable(wscope, [shape[-1], output_depth], tf.float32,tf.random_normal_initializer(stddev=0.02))
    if isDiscriminator:
        tf.add_to_collection('regularizer', tf.contrib.layers.l2_regularizer(scale=0.0001)(weights)) # weight regularizer
    biases = tf.get_variable(bscope, [output_depth], initializer=tf.constant_initializer(0.0))
    out_logit = tf.matmul(net, weights) + biases
    return out_logit

def deconv(net,wscope,bscope,output_shape,receptive_field=[5,5],stride=[2,2]):
    weights = tf.get_variable(wscope, receptive_field+[output_shape[-1], net.get_shape()[-1]],
                                       initializer=tf.random_normal_initializer(stddev=0.02))
    #tf.add_to_collection('regularizer', tf.contrib.layers.l2_regularizer(scale=0.0001)(weights)) # weight regularizer
    net = tf.nn.conv2d_transpose(net, weights, output_shape=output_shape, strides=[1]+stride+[1], padding='SAME')
    biases = tf.get_variable(bscope, [output_shape[-1]], initializer=tf.constant_initializer(0.0))
    net = tf.reshape(tf.nn.bias_add(net, biases), net.get_shape())
    return net

def get_shape(net):
    return net.get_shape().as_list()

def display(img_samples=None, batch_size=None, img_h=None, img_w=None, image_dims=None, step_ind=None):
    
    row = col = int(np.sqrt(batch_size))
    img = np.zeros( [row*img_h, col*img_w] )
    for i in range(row):
        for j in range(col):
            img[i*img_w:(i+1)*img_w, j*img_h:(j+1)*img_h] = img_samples[i*col+j, :].reshape(image_dims[:2])
    #save the result      
    scipy.misc.imsave('multi_{}.jpg'.format(step_ind),img)  

# Discriminator Network

In [5]:
# TODO: add gaussian noise to each layer
def discriminator(inputs, batch_size, is_training=True, reuse=False):
    
    with tf.variable_scope("discriminator", reuse=reuse):
        outsli = []
        
        '''1st: Conv -> lrelu'''
        net = conv(inputs,'d_wconv1','d_bconv1',64, [5,5], [2,2], 'SAME',True) # 28x28 -> 14x14 
        net = tf.nn.leaky_relu(net)
        outsli.append(net)
        
        '''2nd: Conv -> bn -> lrelu'''
        net = conv(net,'d_wconv2','d_bconv2',64*2, [3,3], [2,2], 'SAME', True) # 14x14 -> 7x7
        net = bn(net, scope = 'd_bn2',is_training=is_training)
        net = tf.nn.leaky_relu(net)
        outsli.append(net)
        
        '''3th: Conv -> bn -> lrelu'''
        net = conv(net,'d_wconv3','d_bconv3',64*4, [3,3], [1,1], 'VALID', True) # 7x7 -> 5x5
        net = bn(net, scope = 'd_bn3',is_training=is_training)
        net = tf.nn.leaky_relu(net)
        outsli.append(net)
        
        '''4th: Conv -> bn -> lrelu'''
        net = conv(net,'d_wconv4','d_bconv4',64*4, [3,3], [2,2], 'VALID', True) # 5x5 -> 2x2
        net = bn(net, scope = 'd_bn4',is_training=is_training)
        net = tf.nn.leaky_relu(net)
        outsli.append(net)
        
        net = tf.reshape(net, [batch_size, -1]) # batch_size x (2x2x64x4)

        '''5th: linear '''
        out_logit = linear(net,"d_wlinear5","d_blinear5",11, True) # attention
        '''6th: sigmoid'''
        # out = tf.nn.sigmoid(out_logit)  
        out = outsli

        return out, out_logit

# Generator Network

In [6]:
def generator( z, batch_size, output_height=28, output_width=28,output_depth=1, is_training=True, reuse=False):
    
    height0,width0 = output_height,output_width # 28,28
    height2,width2 = math.ceil(float(height0)/2),math.ceil(float(width0)/2) # 14,14
    height4,width4 = math.ceil(float(height2)/2),math.ceil(float(width2)/2) # 7,7
    height8,width8 = math.ceil(float(height4)/2),math.ceil(float(width4)/2) # 4,4
    height16,width16 = math.ceil(float(height8)/2),math.ceil(float(width8)/2) # 2,2
    
    with tf.variable_scope("generator", reuse=reuse):
        
        '''1st: linear -> reshape -> bn -> relu '''
        net = linear(z, "g_wlinear1", "g_blinear1", 64*height16*width16)
        net = tf.reshape(net,[batch_size, height16, width16, 64]) # -1,2,2,64
        net = bn(net,scope='g_bn1',is_training=is_training)
        net = tf.nn.relu(net)  
    
        '''2nd: deconv -> bn -> relu '''
        output_shape = [batch_size, height8, width8, 64*4] # -1,4,4,64*4
        net = deconv(net,'g_wdeconv2','g_bdeconv2',output_shape,[5,5],[2,2])
        net = bn(net,scope='g_bn2',is_training=is_training)
        net = tf.nn.relu(net)
        
        '''3th: deconv -> bn -> relu '''
        output_shape = [batch_size, height4, width4, 64*2] # -1,7,7,64*2
        net = deconv(net,'g_wdeconv3','g_bdeconv3',output_shape,[5,5],[2,2])
        net = bn(net,scope='g_bn3',is_training=is_training)
        net = tf.nn.relu(net)
       
        '''4th: deconv -> bn -> relu '''
        output_shape = [batch_size, height2, width2, 64*2] # -1,14,14,64
        net = deconv(net,'g_wdeconv4','g_bdeconv4',output_shape,[5,5])
        net = bn(net,scope='g_bn4',is_training=is_training)
        net = tf.nn.relu(net)
        
        '''5th: deconv -> tanh '''
        output_shape = [batch_size, height0, width0, 1*1] # -1,28,28,1
        net = deconv(net,'g_wdeconv5','g_bdeconv5',output_shape,[5,5])
        out = tf.nn.sigmoid(net)

        return out

# set the global parameters.

In [7]:
# some parameters
img_w = 28
img_h = 28
image_dims = [28, 28, 1]
batch_size = 64
z_dim = 128
y_dim = 10
num_class = y_dim + 1
learning_rate = 0.0002 #0.001
beta1 = 0.5
output_height,output_width,output_depth = [28,28,1]

""" Graph Input """
# images
inputs_supervised = tf.placeholder(tf.float32, [batch_size] + image_dims, name='supervised_images')
inputs_unsupervised = tf.placeholder(tf.float32, [batch_size] + image_dims, name='unsupervised_images')

#labels
y = tf.placeholder(tf.float32, [batch_size,y_dim], name='y_label')
# noises
z = tf.placeholder(tf.float32, [batch_size, z_dim], name='z_noise')

flag = tf.placeholder(tf.float32, shape=[], name='flag')

# Loss Function 

different author has different implementation, you should choice one style of them

### below loss function structure borrowed from  
https://github.com/LDOUBLEV/semi-supervised-GAN

In [8]:
""" Loss Function """

'''define the network'''
# output of D for real images
layerlist_real, d_logits_real = discriminator(inputs_supervised, batch_size, is_training=True, reuse=False)

# output of D for fake images
g_img = generator(z, batch_size, is_training=True, reuse=False)
layerlist_fake, d_logits_fake = discriminator(g_img, batch_size, is_training=True, reuse=True)

'''weight normalization for discriminator'''
d_regular = tf.add_n(tf.get_collection('regularizer', 'discriminator'), 'loss')  # D regular loss

    

'''caculate the unsupervised loss'''
unsupervised_label_real = tf.concat([tf.ones_like(y), tf.zeros(shape=(batch_size, 1))], axis=1)
unsupervised_label_fake = tf.concat([tf.zeros_like(y), tf.ones(shape=(batch_size, 1))], axis=1)
# logits_r, logits_f = tf.nn.softmax(d_logits_real), tf.nn.softmax(d_logits_fake)

d_unloss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=unsupervised_label_real*0.9, logits=d_logits_real))
d_unloss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=unsupervised_label_fake*0.9, logits=d_logits_fake))

'''feature match'''
feature_match = tf.constant(0, dtype=tf.float32)
tmp = [tf.reduce_mean(tf.multiply(r-f, r-f))  for r,f in zip(layerlist_real, layerlist_fake)]
for i in tmp: feature_match += i

'''caculate the supervised loss'''
supervised_label = tf.concat([y,tf.zeros(shape=(batch_size, 1))], axis=1)
supervised_loss_real = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=supervised_label, logits=d_logits_real))
supervised_loss_fake = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=unsupervised_label_fake*0.9, logits=d_logits_fake))  # same as d_loss_f

d_loss_1, d_loss_2 = d_unloss_real - d_unloss_fake, supervised_loss_real

''' loss of discriminator'''
supervised_loss = supervised_loss_real + supervised_loss_fake
unsupervised_loss = d_unloss_real + d_unloss_fake

d_loss = unsupervised_loss + supervised_loss_real*flag + d_regular 

''' loss of generator '''
g_loss = d_unloss_fake + 0.5*feature_match





Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### below loss function structure borrowed from    
https://github.com/bruno-31/ImprovedGAN-Tensorflow/blob/master/train_cifar.py and  
https://github.com/openai/improved-gan/blob/master/mnist_svhn_cifar10/train_mnist_feature_matching.py

In [ ]:
""" Loss Function """

'''define the network'''
# output of D for real images
layerlist_real, d_logits_real = discriminator(inputs_supervised, batch_size, is_training=True, reuse=False)

# output of D for fake images
g_img = generator(z, batch_size, is_training=True, reuse=False)
layerlist_fake, d_logits_fake = discriminator(g_img, batch_size, is_training=True, reuse=True)

layerlist_real_unsupervised, d_logits_unsupervised = discriminator(inputs_unsupervised, batch_size, is_training=True, reuse=True)


'''caculate the unsupervised loss'''
unsupervised_label_real = tf.concat([tf.ones_like(y), tf.zeros(shape=(batch_size, 1))], axis=1)
unsupervised_label_fake = tf.concat([tf.zeros_like(y), tf.ones(shape=(batch_size, 1))], axis=1)
# logits_r, logits_f = tf.nn.softmax(d_logits_real), tf.nn.softmax(d_logits_fake)

d_unloss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=unsupervised_label_real*0.9, logits=d_logits_real))
d_unloss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=unsupervised_label_fake*0.9, logits=d_logits_fake))

'''feature match'''
feature_match = tf.constant(0, dtype=tf.float32)
tmp = [tf.reduce_mean(tf.multiply(r-f, r-f))  for r,f in zip(layerlist_real, layerlist_fake)]
for i in tmp: feature_match += i

'''caculate the supervised loss'''
supervised_label = tf.concat([y,tf.zeros(shape=(batch_size, 1))], axis=1)
supervised_loss_real = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=supervised_label, logits=d_logits_real))
supervised_loss_fake = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=unsupervised_label_fake*0.9, logits=d_logits_fake))  # same as d_loss_f


''' loss of discriminator'''

l_unl = tf.reduce_logsumexp(d_logits_unsupervised, axis=1)
l_gen = tf.reduce_logsumexp(d_logits_fake, axis=1)
supervised_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=supervised_label, logits=d_logits_real))
unsupervised_loss = - 0.5 * tf.reduce_mean(l_unl) \
                   + 0.5 * tf.reduce_mean(tf.nn.softplus(l_unl)) \
                   + 0.5 * tf.reduce_mean(tf.nn.softplus(l_gen))

d_loss = unsupervised_loss + supervised_loss

''' loss of generator '''
g_loss = feature_match





split the generator parameters and discriminator parameters into two list, then define how to train the two subnetwork and get the fake image for testing.

In [10]:
""" Training """
# divide trainable variables into a group for D and a group for G
t_vars = tf.trainable_variables()
d_vars = [var for var in t_vars if 'd_' in var.name]
g_vars = [var for var in t_vars if 'g_' in var.name]
# optimizers
d_optim = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(d_loss, var_list=d_vars)
g_optim = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(g_loss, var_list=g_vars)


"""" Testing """
# for test
_, test_logits = discriminator(inputs_supervised, batch_size, is_training=False, reuse=True)
test_labels_pred = tf.nn.softmax(test_logits)
temp = tf.reshape(test_labels_pred[:, -1], shape=[batch_size, 1])
for i in range(10):
    temp = tf.concat([temp, tf.reshape(test_labels_pred[:, -1],shape=[batch_size, 1])], axis=1)
test_labels_pred -= temp
prediction = tf.nn.in_top_k(test_labels_pred, tf.argmax(supervised_label, axis=1), 1)


fake_images = generator(z, batch_size, is_training=False, reuse=True)

# graph inputs for visualize training results
sample_z = np.random.uniform(-1, 1, size=(batch_size , z_dim)).astype(np.float32)

test_labels_onehot = np.zeros([batch_size, y_dim],dtype = np.float32)
test_labels_onehot[np.arange(batch_size), np.arange(batch_size)%int(y_dim)] = 1.0

# Training a GAN

In [11]:
num_steps = 6000
# loop for epoch
start_time = time.time()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step_ind in range(num_steps):
    
    '''get the real data'''
    real_image_batch = mnist.train.next_batch(batch_size)
    batch_images,batch_labels = real_image_batch
    batch_images = batch_images.reshape([batch_size]+image_dims).astype(np.float32)
    batch_labels = real_image_batch[1].astype(np.float32)
    '''get the noise data'''
    batch_z = np.random.uniform(-1, 1, [batch_size, z_dim]).astype(np.float32)
    
    flag_input = 1 if step_ind < 400 else 0 

    feed_dict_input = {inputs_supervised:batch_images,inputs_unsupervised:batch_images, z:batch_z, y:batch_labels, flag:1}
    # update D network
    _ , D_loss, d1, d2 = sess.run([d_optim, d_loss, d_loss_1, d_loss_2], feed_dict=feed_dict_input)
    _ , D_loss, d1, d2 = sess.run([d_optim, d_loss, d_loss_1, d_loss_2], feed_dict=feed_dict_input)

    # update G network
    _, G_loss = sess.run([g_optim,g_loss], feed_dict=feed_dict_input)
   # _, G_loss = sess.run([g_optim,g_loss], feed_dict=feed_dict_input)

    # display training status
    print("Step: [%d] d_loss: %.8f, g_loss: %.8f" % (step_ind, D_loss, G_loss) )

    # save training results for every 300 steps
    if np.mod(step_ind, 300) == 0:
        samples = sess.run(fake_images, feed_dict={z: sample_z})
        display(samples, batch_size, img_h, img_w, image_dims, step_ind)
        
''' testing the di'''       
print('testing................')
count = 0
for i in range(10000//batch_size):
    testx, textl = mnist.test.next_batch(batch_size)
    prediction1 = sess.run(prediction, feed_dict={inputs_supervised:testx, y:textl})
    count += np.sum(prediction1)
print(f'acc: {count/10000}')

Step: [0] d_loss: 2.74950695, g_loss: 1.49454856


/usr/local/python3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Step: [1] d_loss: 1.96254301, g_loss: 1.35227847
Step: [2] d_loss: 1.73136282, g_loss: 1.32268691
Step: [3] d_loss: 1.49738908, g_loss: 1.29308164
Step: [4] d_loss: 1.35347605, g_loss: 1.26671267
Step: [5] d_loss: 1.21242261, g_loss: 1.26091623
Step: [6] d_loss: 1.08511353, g_loss: 1.24323249
Step: [7] d_loss: 1.11811829, g_loss: 1.22308862
Step: [8] d_loss: 1.05652559, g_loss: 1.21816635
Step: [9] d_loss: 1.00498426, g_loss: 1.18879640
Step: [10] d_loss: 0.99508077, g_loss: 1.17240310
Step: [11] d_loss: 0.95415258, g_loss: 1.17435229
Step: [12] d_loss: 0.92159289, g_loss: 1.14264071
Step: [13] d_loss: 0.94192803, g_loss: 1.14142275
Step: [14] d_loss: 0.82619208, g_loss: 1.13413107
Step: [15] d_loss: 0.80396771, g_loss: 1.13649464
Step: [16] d_loss: 0.78793514, g_loss: 1.12904787
Step: [17] d_loss: 0.83301014, g_loss: 1.10376573
Step: [18] d_loss: 0.82380354, g_loss: 1.11703765
Step: [19] d_loss: 0.80326885, g_loss: 1.11349821
Step: [20] d_loss: 0.80454636, g_loss: 1.07388473
Step: [21

NameError: name 'sbatch_size' is not defined